# Generating Safer Workout Routes At Night
By Mina Negahban

Despite the many dangers associated with running at night, 68% of people do so regularly<sup>(1)</sup>. One of the challenges during a run is staying focused, and fears of what could be hiding in the dark can be distracting, and ultimately decrease one's performance. Running should not be stressful or fear inducing, it should be relaxing, or at the very least time a time when you can focus completley on your self and your fitness. Our goal today is to generate safer routes for runners (or anyone out at night), in order to reduce this fear. The route will be optimized based on streetlight locations using Kinetica’s graph solver, to produce the most well lit path between the user’s specified origin and destination points.    

## Required Data Sets
We will be using 2 datasets, which are listed below:
* DC street light data (.csv)
    * https://opendata.dc.gov/datasets/DCGIS::street-lights/about
* DC road network data (.zip)
    * https://download.geofabrik.de/north-america/us/district-of-columbia.html
    * After downloading the .zip file, convert the file named **gis_osm_roads_free_1.shp** into a .csv file using gdal:
        - input ``ogr2ogr -f CSV dcGeo.csv gis_osm_roads_free_1.shp -lco GEOMETRY=AS_WKT`` into your command line

    
Before loading both .csv files into the database, you'll need to get them onto the database machine. Grab the files referenced above and either put them into the /mnt/persist folder of Kinetica Dev Edition, or load them into a directory of your choice (e.g., /safeRoutes) on Kinetica Cloud via the KIFS upload screen. You can find more information here: https://docs.kinetica.com/7.1/tools/kifs/

In [2]:
import gpudb

We'll be interacting with Kinetica along the way, loading data, setting up graph optimziations, and viewing results. Ensure to export these environment variables (or override them below): **KINETICA_HOST, KINETICA_USER, KINETICA_PASS**. All the code below is Python, but many are SQL commands excuted via Python directly against the database you connect to below.

In [ ]:
KINETICA_HOST = os.getenv('KINETICA_HOST', "localhost:9191")
KINETICA_USER = os.getenv('KINETICA_USER', "admin")
KINETICA_PASS = os.getenv('KINETICA_PASS')
db = gpudb.GPUdb(host=KINETICA_HOST, username=KINETICA_USER, password=KINETICA_PASS)

### Ingest Data

In [ ]:
exec_result = db.execute_sql("""
LOAD DATA INTO ki_home.dc_streetlight_fixtures
FROM FILE PATHS 'data/Street_Lights.csv'
FORMAT TEXT (INCLUDES HEADER = true);
""")

In [ ]:
exec_result = db.execute_sql("""
LOAD DATA INTO ki_home.dc_osm
FROM FILE PATHS 'data/dcGeo.csv'
FORMAT TEXT (INCLUDES HEADER = true);
""")

### Prep Streetlight Data

First we must convert the street light points into polygons inorder to create buffers around the lights, which represents their lightspan.

In [ ]:
exec_result = db.execute_sql("""create table dc_buffer_ids
  AS (SELECT st_buffer(ST_POINT(ki_home.dc_streetlight_fixtures.x, ki_home.dc_streetlight_fixtures.y), 9, '', 1), street_light_id
  FROM ki_home.dc_streetlight_fixtures);
  """)


Next we will find all the intersections between the lights and the roads.

In [ ]:
exec_result = db.execute_sql("""CREATE TABLE dc_intersects
  AS (SELECT dc_buffer_ids.street_light_id, dc_osm.osm_id 
FROM dc_buffer_ids, dc_osm 
WHERE ST_Intersects(dc_osm.geom_wkt,dc_buffer_ids.EXPR_0)
);
""")


Convert the polygons representing the street lights into a single multipolygon

In [ ]:
exec_result = db.execute_sql("""create table lights_single_poly
AS (SELECT st_dissolve(EXPR_0)
from dc_buffer_ids
);
""")


Perform gejoin

In [ ]:
exec_result = db.execute_sql("""
create table lightness
AS (SELECT 
  osm_id AS "osm_id",
  COUNT(*) AS "light"
FROM
  ki_home.dc_osm
  JOIN ki_home.lights_single_poly
    ON ST_INTERSECTS(geom_wkt, EXPR_0) = 1
GROUP BY
  osm_id
  );
""")


Add light column

In [ ]:
exec_result = db.execute_sql("""
ALTER TABLE dc_osm
ADD light int;
""")

db.execute_sql("""
update dc_osm os
set light = 1 
where osm_id=
(
SELECT osm_id
        FROM ki_home.lightness l
        WHERE os.osm_id = l.osm_id
        );
""")

### Graph

First we will create the graph with weights

In [ ]:
exec_result = {
  "graph_name": "dc_osm_graph",
  "directed_graph": false,
  "nodes": [],
  "edges": [
    "ki_home.dc_osmosm_small.osm_id AS EDGE_ID",
    "ki_home.dc_osmosm_small.geom_wkt AS EDGE_WKTLINE"
  ],
  "weights": [
    "ki_home.dc_osmosm_small.osm_id AS WEIGHTS_EDGE_ID",
    "ST_Length(ki_home.osm_small.geom_wkt,1)/(ST_NPoints(ki_home.osm_small.geom_wkt)-1) + ((1- ki_home.osm_small.light)*20) AS WEIGHTS_VALUESPECIFIED"
  ],
  "restrictions": [],
  "options": {
    "merge_tolerance": "0.00001",
    "use_rtree": "false",
    "min_x": "-180",
    "max_x": "180",
    "min_y": "-90",
    "max_y": "90",
    "recreate": "true",
    "modify": "false",
    "export_create_results": "false",
    "enable_graph_draw": "true",
    "save_persist": "false",
    "sync_db": "false",
    "add_table_monitor": "false",
    "graph_table": "ki_home.dc_osm_graph_table",
    "add_turns": "false",
    "turn_angle": "60.0",
    "is_partitioned": "false"
  }
}


Now we will solve the graph using the shortest_path solver

In [ ]:
exec_result = {
  "graph_name": "dc_osm_graph",
  "weights_on_edges": [],
  "restrictions": [],
  "solver_type": "SHORTEST_PATH",
  "source_nodes": [
    "{'POINT(-77.037124 38.926142)'} AS NODE_WKTPOINT"
  ],
  "destination_nodes": [
    "{'POINT(-77.042686 38.922676)'} AS NODE_WKTPOINT"
  ],
  "solution_table": "ki_home.dc_lit_path_solved28",
  "options": {
    "export_solve_results": "false",
    "min_solution_radius": "0.0",
    "max_solution_radius": "0.0",
    "max_solution_targets": "0",
    "accurate_snaps": "true",
    "left_turn_penalty": "0.0",
    "right_turn_penalty": "0.0",
    "intersection_penalty": "0.0",
    "sharp_turn_penalty": "0.0",
    "output_edge_path": "false",
    "output_wkt_path": "true"
  }
}

### Further Applications
More layers can be added to this model to generate even safer routes with more nuances. Additional wieghts can be added based on:
* Real time police reports 
    * This will also highlight Kinetica's capabilities with live streaming data ingest
* Cell coverage
* Streets with sidewalks/bikelanes
* CCTV locations
* Zone type
* Terrain type
This model does not have to be limited to only runner safety. Some examples of applications beyond running include:
* Hiking/biking routes
* Student safety on/sorrounding college campuses
* Parking at night

### Refrences

1. Fuehrer, Dan, et al. “What Time of Day Do People Run?” *Runner's World*, 17 Sept. 2020, www.runnersworld.com/training/a20812346/what-time-of-day-do-people-run/. 